## BioInformatics and Machine Learning Intern
Installing pyreadr to read rds file

In [3]:
!pip install pyreadr


Upload the file and view the first five rows of the dataset

In [4]:
import pyreadr
from google.colab import files
import pandas as pd

# Prompt the user to upload an RDS file
uploaded = files.upload()

# Get the first uploaded file (assuming it's an RDS file)
rds_file_path = list(uploaded.keys())[0]

# Use the pyreadr.read_r() function to read the RDS file
result = pyreadr.read_r(rds_file_path)
keys_list = list(result.keys())
data = result[keys_list[0]]  # Use the first key (change as needed)

# Convert the data to a DataFrame
df = pd.DataFrame(data)

df.head()

Saving exgr_test.rds to exgr_test (1).rds


,transcript_id,transcript_name,sequence_names,start,end,width,strand,exon_id,exon_name,rank
0,1,ENST00000456328.2,chr1,11869,12227,359,+,1,ENSE00002234944.1,1
1,1,ENST00000456328.2,chr1,12613,12721,109,+,5,ENSE00003582793.1,2
2,1,ENST00000456328.2,chr1,13221,14409,1189,+,8,ENSE00002312635.1,3
3,2,ENST00000450305.2,chr1,12010,12057,48,+,2,ENSE00001948541.1,1
4,2,ENST00000450305.2,chr1,12179,12227,49,+,3,ENSE00001671638.2,2


## Task 1: Counting Unique Transcripts
To find the count of unique transcripts, we identify the unique transcript IDs within the dataset. Each transcript is uniquely represented by its transcript ID

In [5]:
unique_transcripts = df['transcript_id'].nunique()
unique_transcripts


234485

As you can see there are 234485 transcripts in the entire datatset

##Task 2: Counting Unique Exons
### To determine the number of unique exons, we follow these steps:



1.  Group the data by transcript IDs.
2.  Calculate the count of exons within each transcript.
3. Sum up the counts from all transcripts to obtain the total count of unique exons.



In [6]:
# Group the DataFrame by transcript IDs
transcript_groups = df.groupby('transcript_id')

# Calculate the count of exons within each transcript
exon_counts_per_transcript = transcript_groups.size()

# Sum up the counts to find the total count of unique exons
total_unique_exons = exon_counts_per_transcript.sum()

# Display the total count of unique exons
total_unique_exons


1460986

There are 140986 total unique exons in the entire dataset.

### Task 3: Calculating Average and Median Exon Length
To calculate the average and median length of an exon, we utilize the 'mean' and 'median' functions applied to the 'width' column, which represents the exon width.



In [7]:
average_length = df['width'].mean()
print(average_length)

median_length = df['width'].median()
print(median_length)

262.9930635885628
130.0


The aveage length and median of an exon are 262.99 and 130 units respectively

## Task 4: Calculating the length of introns

Inorder to calculate the length of introns we need to handle the positive strands and negative strands differently because

Length of intron = Start-co-ordinate of exon - end-co-ordinate of previous exon


For positive strands,the end coordinate of previous exon would be the end cordinate of the above row

For negative strands,the end coordinate of previos exon would be end coordinate of the row below


In [20]:
# Separate the DataFrame into positive and negative strand DataFrames
positive_strand_df = df[df['strand'] == '+'].copy()
negative_strand_df = df[df['strand'] == '-'].copy()

# Sort the positive strand DataFrame by 'transcript_id' and 'rank'
positive_strand_df = positive_strand_df.sort_values(by=['transcript_id', 'rank'])

# Calculate intron lengths for positive strand exons
#shift(1) shifts that column by one row below In this way we get the above end-coordinate
positive_strand_df['intron_length'] = (
    positive_strand_df['start'] - positive_strand_df.groupby('transcript_id')['end'].shift(1)
) - 1

# Sort the negative strand DataFrame by 'transcript_id' and 'rank'
negative_strand_df = negative_strand_df.sort_values(by=['transcript_id', 'rank'])


# Calculate intron lengths for negative strand exons
#shift(-1) shifts that column by one row aboce In this way we get the below end-coordinate
negative_strand_df['intron_length'] = (
    negative_strand_df['start'] - negative_strand_df.groupby('transcript_id')['end'].shift(-1)
) - 1

# Concatenate positive and negative strand DataFrames row-wise
combined_df = pd.concat([positive_strand_df, negative_strand_df], axis=0)

# Fill NaN values in the 'intron_length' column with 0 and convert to integer
combined_df['intron_length'] = combined_df['intron_length'].fillna(0).astype(int)

# Sort the combined DataFrame by 'transcript_id' and 'rank'
combined_df = combined_df.sort_values(by=['transcript_id', 'rank'])
combined_df

,transcript_id,transcript_name,sequence_names,start,end,width,strand,exon_id,exon_name,rank,intron_length
0,1,ENST00000456328.2,chr1,11869,12227,359,+,1,ENSE00002234944.1,1,0
1,1,ENST00000456328.2,chr1,12613,12721,109,+,5,ENSE00003582793.1,2,385
2,1,ENST00000456328.2,chr1,13221,14409,1189,+,8,ENSE00002312635.1,3,499
3,2,ENST00000450305.2,chr1,12010,12057,48,+,2,ENSE00001948541.1,1,0
4,2,ENST00000450305.2,chr1,12179,12227,49,+,3,ENSE00001671638.2,2,121
...,...,...,...,...,...,...,...,...,...,...,...
1460981,234481,ENST00000387409.1,chrM,5826,5891,66,-,760545,ENSE00001544488.1,1,0
1460982,234482,ENST00000387416.2,chrM,7446,7514,69,-,760546,ENSE00001544487.2,1,0
1460983,234483,ENST00000361681.2,chrM,14149,14673,525,-,760547,ENSE00001434974.2,1,0
1460984,234484,ENST00000387459.1,chrM,14674,14742,69,-,760548,ENSE00001544476.1,1,0


## Bonus Task - Calculate L1, L2, U1, U2 ensuring no overlap

Inorder to calculate l1, l2, u1, u2 for each exon(except the first and last exon) we need both the intron length on left side (that is the current intron length) and the intron length on right side (that is the intron length of next exon)

So first lets create a column which also stores the intron length of the next exon

As for calculating intron length we need to divide the dataset into 2 parts (one for positive strands and one for negative strands)

The same way we will divide it and calculate the next intron length

In [21]:

# For negative strand DataFrame:
# Calculate the previous intron length within each transcript group
negative_strand_df['next_intron_length'] = negative_strand_df.groupby('transcript_id')['intron_length'].shift(1)

# Fill missing values (NaN) with 0 and convert the column to integers
negative_strand_df['next_intron_length'] = negative_strand_df['next_intron_length'].fillna(0).astype(int)

# For positive strand DataFrame:

# Calculate the previous intron length within each transcript group
positive_strand_df['next_intron_length'] = positive_strand_df.groupby('transcript_id')['intron_length'].shift(-1)

# Fill missing values (NaN) with 0 and convert the column to integers
positive_strand_df['next_intron_length'] = positive_strand_df['next_intron_length'].fillna(0).astype(int)


In [22]:
complete_df=  pd.concat([positive_strand_df, negative_strand_df], axis=0)

# Fill NaN values in the 'intron_length' column with 0 and convert to integer
complete_df['intron_length'] = complete_df['intron_length'].fillna(0).astype(int)

# Sort the combined DataFrame by 'transcript_id' and 'rank'
complete_df = complete_df.sort_values(by=['transcript_id', 'rank'])

copy_of_complete_dataframe=complete_df

If the length, next length and width are greater than 200 then calculating them is much easier. We simply use the below formulas

1. L1= start_coordinate - 100
2. L2= start_cordinate + 100
3. u1= end_cordinate - 100
4. u2= end_cordinate + 100

Lets consider each one and understand what will happen if these (length, next length or width are greater than 200)

1. L1
     
        According to the question L1 is 100 units of length before sj (start-cordinate) Hence L1 = start - 100
        If we take the first exon L1 would be zero Since transcripts start only with an exon there is no space for l1
        So l1 basicaly depends on the intron length If intron length is 0 then L1=0 I have add the code for this statement on
        line 14
        But if intron length is less than 200 then l1 will become start minus half of intron length L1=start-intron_length/2

2. L2
     
        According to the question L2 is 100 units of length after sj (start-cordinate) Hence L2 = start + 100
        So l2 basicaly depends on the exon width
        If width less than 200 then l2 will become start plus half of width L2=start+exon_width/2

3. u1
     
        According to the question u1 is 100 units of length before sj(end-cordinate) Hence u1 = end - 100
        So u1 also basicaly depends on the exon width
        If width less than 200 then u1 will become end minus half of exon width L2=end - exon_width/2

4. u2
        According to the question u2 is 100 units of length after  sj(end-cordinate) Hence u2 = end +100
        If we take the last exon u2 would be zero Since transcripts end only with an exon there is no space for u2
        So u2 basicaly depends on the next intron length If next intron length is 0 then u2=0 I have add the code for the same
        But if next intron length is less than 200 then u2 will become end minus half of next intron length L2=start-intron_length/2



###### In case if you want the dataframe to not contain values where L2=u1 then please uncomment the lines. It will result in the dataframe where L1 cannot be equal to u1.

In [11]:

import math

def calculate_positions(exon):
    # Extract data from the exon
    start = exon['start']
    end = exon['end']
    length = exon['intron_length']  # Calculate the length of the exon
    width = exon['width']
    next_intron_length = exon['next_intron_length']

    def calculate_l1(start, length):
        # Calculate L1 based on the exon's length
        if length <= 200:
            if length == 0:
                l1 = 0
            else:
                if length % 2 == 0:
                    length = length - 1
                # else:
                #     length = length - 2

                l1 = start - (length / 2)

        else:
            l1 = start - 100

        return l1

    def calculate_l2(start, width):
        # Calculate L2 based on the exon's width
        if width <= 200:
            if width % 2 == 0:
                width = width - 1
            # else:
            #     width = width - 2

            l2 = start + (width / 2)
        else:
            l2 = start + 100

        return l2

    def calculate_u1(end, width):
        # Calculate U1 based on the exon's width
        if width <= 200:
            if width % 2 == 0:
                width = width - 1
            # else:
            #     width = width - 2

            u1 = end - (width / 2)
        else:
            u1 = end - 100

        return u1

    def calculate_u2(end, next_intron_length):
        # Calculate U2 based on the next_intron_length
        if next_intron_length != 0:
            if next_intron_length <= 200:
                if next_intron_length % 2 == 0:
                    next_intron_length = next_intron_length - 1
                # else:
                #     next_intron_length = next_intron_length - 2

                u2 = end + (next_intron_length / 2)
            else:
                u2 = end + 100
        else:
            u2 = 0

        return u2

    # Calculate the positions
    l1 = calculate_l1(start, length)
    l2 = calculate_l2(start, width)
    u1 = calculate_u1(end, width)
    u2 = calculate_u2(end, next_intron_length)

    # Round the positions using math functions
    l1 = math.ceil(l1)
    l2 = math.floor(l2)
    u1 = math.ceil(u1)
    u2 = math.floor(u2)

    return pd.Series({'L1': l1, 'L2': l2, 'U1': u1, 'U2': u2})

# Apply the calculate_positions function to the DataFrame and create new columns
complete_df[['L1', 'L2', 'U1', 'U2']] = complete_df.apply(
    lambda exon: calculate_positions(exon),
    axis=1
)

# Sort the combined DataFrame by 'transcript_id' and 'rank'
complete_df = complete_df.sort_values(by=['transcript_id', 'rank'])


## Optimized code for Bonus problem

Below is the optimized version of the bonus problem.

In [23]:

import math
import pandas as pd
import numpy as np

def calculate_positions_vectorized(df):
    # Calculate L1, L2, U1, and U2 for the entire DataFrame
    half_intron_length = df['intron_length'].apply(lambda x: x - 1 if x % 2 == 0 else x) / 2
    half_exon_length = df['width'].apply(lambda x: x - 1 if x % 2 == 0 else x) / 2
    next_intron_length=df['next_intron_length'].apply(lambda x: x - 1 if x % 2 == 0 else x) / 2

    df['L1'] = np.where(df['intron_length'] == 0, 0, (df['start'] - half_intron_length.clip(upper=100)).clip(lower=0).apply(math.ceil))
    df['L2'] = (df['start'] + half_exon_length.clip(upper=100)).apply(math.floor)
    df['U1'] = (df['end'] - half_exon_length.clip(upper=100)).apply(math.ceil)
    df['U2'] = np.where(df['next_intron_length'] == 0, 0, (df['end'] + next_intron_length.clip(upper=100)).clip(lower=0).apply(math.floor))



    return df

# Apply the calculate_positions_vectorized function to the DataFrame
final_df = calculate_positions_vectorized(copy_of_complete_dataframe)

# Sort the combined DataFrame by 'transcript_id' and 'rank'
final_df = final_df.sort_values(by=['transcript_id', 'rank'])


In [24]:
final_df.head(10)

,transcript_id,transcript_name,sequence_names,start,end,width,strand,exon_id,exon_name,rank,intron_length,next_intron_length,L1,L2,U1,U2
0,1,ENST00000456328.2,chr1,11869,12227,359,+,1,ENSE00002234944.1,1,0,385,0,11969,12127,12327
1,1,ENST00000456328.2,chr1,12613,12721,109,+,5,ENSE00003582793.1,2,385,499,12513,12667,12667,12821
2,1,ENST00000456328.2,chr1,13221,14409,1189,+,8,ENSE00002312635.1,3,499,0,13121,13321,14309,0
3,2,ENST00000450305.2,chr1,12010,12057,48,+,2,ENSE00001948541.1,1,0,121,0,12033,12034,12117
4,2,ENST00000450305.2,chr1,12179,12227,49,+,3,ENSE00001671638.2,2,121,385,12119,12203,12203,12327
5,2,ENST00000450305.2,chr1,12613,12697,85,+,4,ENSE00001758273.2,3,385,277,12513,12655,12655,12797
6,2,ENST00000450305.2,chr1,12975,13052,78,+,6,ENSE00001799933.2,4,277,168,12875,13013,13014,13135
7,2,ENST00000450305.2,chr1,13221,13374,154,+,7,ENSE00001746346.2,5,168,78,13138,13297,13298,13412
8,2,ENST00000450305.2,chr1,13453,13670,218,+,9,ENSE00001863096.1,6,78,0,13415,13553,13570,0
9,3,ENST00000473358.1,chr1,29554,30039,486,+,10,ENSE00001947070.1,1,0,524,0,29654,29939,30139


## Overlapping condition for Bonus problem



One overlapping conditon would be when u2(current exon) > L1 (next exon)

The code below this cells iterates over the dataframe and checks wether is there any row which has u2(current exon) > L1 (next exon) If yes then it prints the row

In [15]:
# Assuming 'complete_df' is the DataFrame you're working with
df = final_df

# Group the DataFrame by 'transcript_id'
groups = df.groupby('transcript_id')

# Initialize an empty list to store DataFrames that meet the conditions
result_dfs = []

# Iterate through each group
for _, group in groups:
    # Check the strand information to determine which shift to use
    if group['strand'].iloc[0] == '+':
        mask = group['U2'].gt(group['L1'].shift(-1))
    else:
        mask = group['U2'].gt(group['L1'].shift(1))

    # Append the DataFrame that meets the conditions to the list
    result_dfs.append(group[mask])

# Concatenate the resulting DataFrames
result_df = pd.concat(result_dfs)

# 'result_df' contains the concatenated DataFrames that meet the conditions
result_df

,transcript_id,transcript_name,sequence_names,start,end,width,strand,exon_id,exon_name,rank,intron_length,next_intron_length,L1,L2,U1,U2


Since the above dataframe is empty it simply means that there is no overlap

However there also be one more condition of overlap where L2>u1 of the same exon The below code iterates over the dataframe and checks for any such conditon

In [16]:
df = final_df

# Group the DataFrame by 'transcript_id'
transcript_groups = df.groupby('transcript_id')

# Initialize an empty list to store DataFrames that meet the conditions
resulting_dataframes = []

# Iterate through each group
for _, transcript_group in transcript_groups:
    # Check if 'L2' is greater than 'U1' within the group
    condition_mask = transcript_group['L2'].gt(transcript_group['U1'])

    # Append the DataFrame that meets the conditions to the list
    resulting_dataframes.append(transcript_group[condition_mask])

# Concatenate the resulting DataFrames
result_l2_u1 = pd.concat(resulting_dataframes)

# 'result_l2_u1' contains the concatenated DataFrames that meet the conditions
print(result_l2_u1)


Empty DataFrame
Columns: [transcript_id, transcript_name, sequence_names, start, end, width, strand, exon_id, exon_name, rank, intron_length, next_intron_length, L1, L2, U1, U2]
Index: []


Since the above dataframe is empty it simply means that there is no overlap. There are case where L2=u1 when width is slightly greater than 200. I have not handled those cases but the only solution can be is do l2 = l2 -1 so that it wont be equal to u1 I have not written the code for that beacause I am unsure its required or not but coding that part wont take long.